In [1]:
import numpy as np
import theano
import theano.tensor as T

Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN 4007)


In [4]:
a = np.array([[1, 2, 3, 4], [1, 2, 3, 4], [2, 3, 4, 4], [4, 5, 6, 4]])
b = np.array([i * np.identity(4) for i in range(3)])
print b

[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 1.  0.  0.  0.]
  [ 0.  1.  0.  0.]
  [ 0.  0.  1.  0.]
  [ 0.  0.  0.  1.]]

 [[ 2.  0.  0.  0.]
  [ 0.  2.  0.  0.]
  [ 0.  0.  2.  0.]
  [ 0.  0.  0.  2.]]]


In [9]:
print np.einsum("ijk, xky->ijxy", a[None, :, :], b)[0].reshape(12, 4)

[[  0.   0.   0.   0.]
 [  1.   2.   3.   4.]
 [  2.   4.   6.   8.]
 [  0.   0.   0.   0.]
 [  1.   2.   3.   4.]
 [  2.   4.   6.   8.]
 [  0.   0.   0.   0.]
 [  2.   3.   4.   4.]
 [  4.   6.   8.   8.]
 [  0.   0.   0.   0.]
 [  4.   5.   6.   4.]
 [  8.  10.  12.   8.]]


In [84]:
embeddings_matrix = theano.shared(numpy.array([[1, 2, 3, 4], [1, 2, 3, 4], [2, 3, 4, 4], [4, 5, 6, 4]], dtype=theano.config.floatX))
index_matrix = T.imatrix()
r_index_matrix = T.imatrix()

In [121]:
role = np.array([i * np.identity(4) for i in range(3)])
print role

[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 1.  0.  0.  0.]
  [ 0.  1.  0.  0.]
  [ 0.  0.  1.  0.]
  [ 0.  0.  0.  1.]]

 [[ 2.  0.  0.  0.]
  [ 0.  2.  0.  0.]
  [ 0.  0.  2.  0.]
  [ 0.  0.  0.  2.]]]


In [122]:
r_emb = theano.shared(numpy.array(role, dtype=theano.config.floatX)) 

In [196]:
x = T.tensordot(embeddings_matrix[None, :, :], r_emb, axes=[[2], [1]]).flatten(4)#.flatten(1)

In [197]:
f = theano.function([], x)

In [198]:
output = f()

In [199]:
test_e = [[1, 2, 3, 4], [1, 2, 3, 4], [2, 3, 4, 4], [4, 5, 6, 4]]
for i in test_e:
    for j in range(3):
        print np.array(i).dot(role[j])

[ 0.  0.  0.  0.]
[ 1.  2.  3.  4.]
[ 2.  4.  6.  8.]
[ 0.  0.  0.  0.]
[ 1.  2.  3.  4.]
[ 2.  4.  6.  8.]
[ 0.  0.  0.  0.]
[ 2.  3.  4.  4.]
[ 4.  6.  8.  8.]
[ 0.  0.  0.  0.]
[ 4.  5.  6.  4.]
[  8.  10.  12.   8.]


In [200]:
print output

[[[[  0.   0.   0.   0.]
   [  1.   2.   3.   4.]
   [  2.   4.   6.   8.]]

  [[  0.   0.   0.   0.]
   [  1.   2.   3.   4.]
   [  2.   4.   6.   8.]]

  [[  0.   0.   0.   0.]
   [  2.   3.   4.   4.]
   [  4.   6.   8.   8.]]

  [[  0.   0.   0.   0.]
   [  4.   5.   6.   4.]
   [  8.  10.  12.   8.]]]]


In [185]:
y = x[index_matrix * 3 + r_index_matrix]

In [138]:
f2 = theano.function(inputs=[index_matrix, r_index_matrix], y)

SyntaxError: non-keyword arg after keyword arg (<ipython-input-138-30348642c33d>, line 1)

In [134]:
inp = [[1, 2]]

In [139]:
1410 * 3

4230

In [135]:
out = f2(numpy.array([inp, inp], dtype=numpy.int32))

TypeError: ('Bad input argument to theano function with name "<ipython-input-129-dd58058e82e3>:1"  at index 0(0-based)', 'Wrong number of dimensions: expected 2, got 3 with shape (2, 1, 2).')

In [136]:
xx = embeddings_matrix[index_matrix] * r_emb[r_ind]
f3 = theano.function(inputs=[r_ind, index_matrix], x * y)

SyntaxError: non-keyword arg after keyword arg (<ipython-input-136-210c1bda78a8>, line 2)

In [137]:
print x.broadcastable

(False, False)


In [41]:
print y.broadcastable

(False, False, False)


In [43]:

out = f([[0, 0], [1, 1]])
out2 = f2([[0, 0], [1, 2]])
z = T.dot(y[:, :, :, None], x)
f3 = theano.function([r_ind, index_matrix], z)

In [47]:
print z.broadcastable

(False, False, False, False, False, False)


In [48]:
out3 = f3([[0, 0], [1, 1]])

TypeError: Missing required input: <TensorType(int32, matrix)>